In [218]:
import qiskit as qk
import numpy as np
import scipy.linalg as lin
import math
from qiskit.quantum_info import Statevector , Operator , partial_trace , DensityMatrix
from qiskit.circuit.library.standard_gates import HGate , XGate
from qiskit.circuit.library import GlobalPhaseGate
from qiskit.extensions import Initialize , UnitaryGate
from qiskit.providers.aer import AerSimulator
import sys
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit import QuantumCircuit, Aer, transpile
import importlib
import PMR_LCU
importlib.reload(PMR_LCU)                                   # Reload to reflect updates
from PMR_LCU import *

/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:99: SyntaxWarning: invalid escape sequence '\o'
  """
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:267: SyntaxWarning: invalid escape sequence '\o'
  """"
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:299: SyntaxWarning: invalid escape sequence '\o'
  """"
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:338: SyntaxWarning: invalid escape sequence '\o'
  """


## Parameters of the system

Here, we define the parameters for the simulation of the Hamiltonian $H(t) = \sum_{i=1}^n h_i Z_i + V_x \sum_{i=1}^{n-1} Z_i Z_{i+1} + cos(\omega t) \sum_{i=1}^{n-1} c_i X_i X_{i+1}$.  

In [313]:
#====================== Circuit Paramters ================
number_of_spins = 2    # number of particles (qubits)
K = 2                  # only two modes -w and w
M = number_of_spins-1  # number of permutation operators

C0 = 1.0               # Parameter for the floquet interaction strength
Coeffs = C0*np.array([2+(-1)**x for x in np.arange(1,number_of_spins)]) # there are n elements in this array
Omega = 0.0
h0 = 1.0
Longit_h = h0*np.array([1.5 + 0.5*(-1)**x for x in range(number_of_spins)])
Longit_h = np.array([1. , 2.])
Vx = [0.0]*M
C = np.max(Coeffs)

Gammas_i = Coeffs
Gammas_k = [0.5]*K
Qmax = 2

Gamma_1 = np.sum(Gammas_i)
Gamma_2 = np.sum(Gammas_k)
Gamma = Gamma_2 * Gamma_1
Gamma_list = [Gammas_i , Gammas_k]

if Gamma < 0.5:
    Delta_t = np.log(2)
else:
    Delta_t = np.log(2)/(Gamma)
GDt = Gamma*Delta_t

sfactor = np.sqrt(1 + GDt + GDt**2)

NumberOfTimeSteps = 1
FinalTime = NumberOfTimeSteps*Delta_t

print(f'The simulation parameters are ... ')
print(f'GDt is {GDt} Gamma = {Gamma} V={Vx} hs = {Longit_h} Gamma_list = {Gamma_list} Delta_t = {Delta_t} time = {FinalTime} Omega = {Omega} Number of time steps = {NumberOfTimeSteps}')

The simulation parameters are ... 
GDt is 0.6931471805599453 Gamma = 1.0 V=[0.0] hs = [1. 2.] Gamma_list = [array([1.]), [0.5, 0.5]] Delta_t = 0.6931471805599453 time = 0.6931471805599453 Omega = 0.0 Number of time steps = 1


### Reading in exact numerical results

In [137]:
InitialStateString = '0'*number_of_spins
# Read in the theoretical (exact numerical) values of the final state, given the initial state!
import json 

# with open("./Exact_Results/FinalState_params1_numoftimestep_"+str(NumberOfTimeSteps)+"_init_"+InitialStateString+".json", "r") as f:
    # data = json.load(f)

with open("./Exact_Results/FinalState_testparV=0_numoftimestep_"+str(NumberOfTimeSteps)+"_init_"+InitialStateString+".json", "r") as f:
    data = json.load(f)

# Extract the first term (complex number) from each entry
complex_numbers = []
for entry in data:
    # Extract everything before the first comma (first term)
    first_term = entry.split(",")[0].strip(" {}")  # Remove spaces and braces
    # Replace " I" with "j" (Python complex notation)
    first_term = first_term.replace(" ", "").replace("I", "j")
    # Append to list
    complex_numbers.append(first_term)

# Convert strings to complex numbers
FinalStateExact = np.array([complex(num) for num in complex_numbers], dtype=np.complex128)
#FinalStateExact = [FinalStateExact[0] , 0 , 0 , FinalStateExact[1]]
print(f'The exact vector is {FinalStateExact}')

The exact vector is [-0.581952+0.771491j  0.      +0.j        0.      +0.j
  0.      -0.257164j]


### Initializing the Quantum circuit

In [314]:
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + Qmax + 1

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>')
qQubits = qk.QuantumRegister(Qmax , '|q>')
ancQubit = qk.QuantumRegister(1 , '|anc>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
q_qbits_index = 3
anc_qbits_index = 4

# =========================== Building the circuits  U_0(dt) U_od(dt) ... U_0(dt) U_od(dt) =================================== #

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )
FullCircwithPS = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )


numberofsteps = FinalTime
# Prepare_full_unitary( FullCirc , [Longit_h , Vx] , Omega , Delta_t , Gamma_list , [kq_qbits_index , iq_qbits_index , z_qbits_index , q_qbits_index , anc_qbits_index] , NumberOfTimeSteps )


Prepare_full_unitary_wo_Rgate( FullCircwithPS , [Longit_h , Vx] , Omega , Delta_t , Gamma_list , [kq_qbits_index , iq_qbits_index , z_qbits_index , q_qbits_index , anc_qbits_index] , NumberOfTimeSteps )

# FullCircwithPS.ry(2*GDt ,iqQubits[0])
# FullCircwithPS.cx(iqQubits[0] , qQubits[0])

# Uc_Phi(FullCircwithPS , M , [Longit_h , Vx] , Delta_t , Qmax , iq_qbits_index , z_qbits_index , q_qbits_index , False)

# FullCircwithPS.cx(iqQubits[0] , qQubits[0]) 
# FullCircwithPS.ry(-2*GDt ,iqQubits[0])


#backend_sim = Aer.get_backend('qasm_simulator')
#transpiled_circuit = transpile(FullCircwithPS, backend_sim)

#print(FullCircwithPS.draw())
#Ops = SparsePauliOp.from_list([('I'*(Nkq + Niq)+'XIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IXII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'ZIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IZII'+'I'*Q , 1)])
#print(f'Circuit Prepared! The parameters are:  Vx = {Vx} , hs = {Longit_h} , Omega = {Omega} , C0 = {C0} , Q = {Q} , Gamma = {Gam} , Delta_t = {Delta_t} , GDt = {Gam*Delta_t}') 

### State initialization and simulation

In [316]:
# Initializing the |z> state:
InitStateZ = Statevector.from_label('0'*number_of_spins)
InitStateZ += Statevector.from_label('1'*number_of_spins)
InitStateZ = InitStateZ/np.linalg.norm(InitStateZ)

print(f'The initial state is {InitStateZ}')

# Making the full system state:
InitState = Statevector.from_label('0'*Nkq)
InitState = Statevector.from_label('0'*Niq).tensor(InitState)
InitState = InitStateZ.tensor(InitState)
InitState = Statevector.from_label('0'*Qmax).tensor(InitState)
InitState = Statevector.from_label('0').tensor(InitState)

FinalStateFull = InitState.evolve(FullCirc)
FinalStatewoRgate = InitState.evolve(FullCircwithPS)

def state_overlap( state1 , state2):
    l1 = len(state1)
    l2 = len(state2)
    # print(f'l1 is {l1} and l2 is {l2}')
    if l1 != l2:
        ValueError("The dimensions of the states don't match!")
    else:
        return np.dot(np.conjugate(state1) , state2)

def post_select_state(TotalState , NumOfPreviousQubits , NumOfAfterQubits):
    """ 
    TotalState is the state |kq> tensor |iq> tensor |z> tensor |q> tensor |ancilla>
    NumOfPreviousQubits specifies the number of qubits that are on the registers previous to the z register.
    NumOfAfterQubits specifies the number of qubits that are on the registers after the z register.
    """
    increment = 2**( NumOfPreviousQubits )
    HighestMultiple = int( len(TotalState) / increment)
    StateHighCut = np.array( [TotalState[i*increment] for i in range(HighestMultiple)] )

    aftercut = 2**(NumOfAfterQubits)
    return StateHighCut[0:int(len(StateHighCut)/aftercut)]

FinalStatePS = post_select_state( FinalStatewoRgate , Niq + Nkq , Qmax+1 )
NormalizationPS = np.linalg.norm( FinalStatePS )
FinalStatewithR = post_select_state( FinalStateFull , Niq + Nkq , Qmax+1 )

print( f'The normalization factor is {NormalizationPS}' )
print( f'The Post selected state is {FinalStatePS/NormalizationPS}' )
print( f'The overlap of the Post selected with the exact is {state_overlap(FinalStatePS , FinalStateExact)}' )
print( f'The overlap of the Post selected with the exact is {state_overlap(FinalStatewithR , FinalStateExact)}' )
# a = (1-1.0j*GDt*np.exp(-3.0j*(Delta_t)))
# b = (1-1.0j*GDt*np.exp(3.0j*(Delta_t)))
# n = np.sqrt(np.abs(a)**2 + np.abs(b)**2)
# print(f'The number is {-1.0j*np.sqrt(GDt)/(np.sqrt(1 + GDt + GDt**2/2.0))}')
# print( f'The expected coefficients are  {1.0/np.sqrt(1 + GDt**2)} and {a/n} and {b/n}' )


The initial state is Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j],
            dims=(2, 2))
The normalization factor is 0.6228322506164639
The Post selected state is [-0.39448369-0.82977001j  0.        +0.j          0.        +0.j
 -0.39448369+0.01571536j]
l1 is 4 and l2 is 4
The overlap of the Post selected with the exact is (-0.25824552546764556-0.42712585592884067j)
l1 is 4 and l2 is 4
The overlap of the Post selected with the exact is (-0.41150220552507366+0.3636841094473334j)


In [96]:
nlow = 3
nhigh = 5
a = Statevector.from_label('0'*nlow + '11' + '0'*nhigh)

increment = 2**( nhigh )
HighestMultiple = int( 2**( nlow + nhigh + 2 ) / increment)
print(f'The highest multiple is {HighestMultiple}')
b = np.array( [a[i*increment] for i in range(HighestMultiple)] )

lowcut = 2**(nlow)
c = b[0:int(len(b)/lowcut)]
c = c/np.linalg.norm( c )

print(b)
print(c)

The highest multiple is 32
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [162]:
# Transpiling the circuit and getting a gate count:
backend_sim = Aer.get_backend('qasm_simulator')
transpiled_circuit = transpile(FullCirc, backend_sim)

gate_count = transpiled_circuit.count_ops()
gate_depth = transpiled_circuit.depth()
Toffoli_count = gate_count.get('ccx' , 0)
CNOT_count = gate_count.get('cx' , 0)
#print("\nGate Count:", gate_count)
print(f'The gate depth is {gate_depth} , total number of qubits required is {Ntotal}, the number of Toffoli gates is {Toffoli_count}, and number of CNOTs is {CNOT_count}')

#print(FullCirc.draw())

The gate depth is 308 , total number of qubits required is 10, the number of Toffoli gates is 36, and number of CNOTs is 158


In [163]:
IBMProvider.save_account(token='1a6f8188ff1654f78d9c4418c7382addf037bf1aca1ec8a40fe866c40123a30913609310721808e16cf25f3f5f92de14f163d30a8162b5a0bba7bda1324ca1cc', overwrite=True )
provider = IBMProvider(instance="usc/hen-research-g/hen-lab")

backend = provider.get_backend('ibm_strasbourg')
Estimator = BackendEstimator(backend)
job = Estimator.run( [FullCirc]*len(Ops) , [Ops[i] for i in range(len(Ops))]  , shots = 5000 )
id=job.job_id()
print(f'The job id is {id}')

The job id is 3978659f-249a-4f61-bd27-78e21a3ca017


In [164]:
results = job.result()
#counts = results.get_counts()


In [165]:
values = results.values
metadata = results.metadata
print(f'The values are {values} and the metadata is {metadata}')

The values are [ 0.0176  0.138   0.942  -0.1512] and the metadata is [{'variance': 0.99969024, 'shots': 5000}, {'variance': 0.980956, 'shots': 5000}, {'variance': 0.11263600000000007, 'shots': 5000}, {'variance': 0.97713856, 'shots': 5000}]


### Tests

#### Convention Test

In [156]:
# Convention tests:
number_of_spins = 2
Qmax = 2
K = 2
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + Qmax + 1

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>')
qQubits = qk.QuantumRegister(Qmax , '|q>')
ancQubit = qk.QuantumRegister(1 , '|anc>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
q_qbits_index = 3
anc_qbits_index = 4

# =========================== Building the circuits  U_0(dt) U_od(dt) ... U_0(dt) U_od(dt) =================================== #

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )
FullCircwithPS = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )

FullCirc.x(qQubits[0])
#FullCirc.x(zQubits[1])

# Initializing the |z> state:
InitStateZ = Statevector.from_label('0'*number_of_spins)

# Making the full system state:
InitState = Statevector.from_label('0'*Nkq)
InitState = Statevector.from_label('0'*Niq).tensor(InitState)
InitState = InitStateZ.tensor(InitState)
InitState = Statevector.from_label('0'*Qmax).tensor(InitState)
InitState = Statevector.from_label('0').tensor(InitState)

finalstate = InitState.evolve(FullCirc)
finalstate_index = [i for i in range(len(finalstate)) if finalstate[i]!=0 ]
print(finalstate_index)


[64]


#### Permutation $U_{cP}$ test

In [169]:
number_of_spins = 2
Qmax = 2
K = 2
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + Qmax + 1

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>')
qQubits = qk.QuantumRegister(Qmax , '|q>')
ancQubit = qk.QuantumRegister(1 , '|anc>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
q_qbits_index = 3
anc_qbits_index = 4

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )

#Uc_P( FullCirc , 1 , 1 , 2 , 1 , False )
Uc_P( FullCirc , 1 , 1 , 2 , 2 , True )

print(FullCirc.draw())

                
|kq>_0: ────────
                
|kq>_1: ────────
                
|iq>_0: ────────
                
|iq>_1: ───■────
        ┌──┴───┐
 |z>_0: ┤0     ├
        │  cXX │
 |z>_1: ┤1     ├
        └──────┘
 |q>_0: ────────
                
 |q>_1: ────────
                
 |anc>: ────────
                


#### State preparation test

In [3]:
number_of_spins = 2
Qmax = 2
K = 2
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + Qmax + 1

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>')
qQubits = qk.QuantumRegister(Qmax , '|q>')
ancQubit = qk.QuantumRegister(1 , '|anc>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
q_qbits_index = 3
anc_qbits_index = 4

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )
B_state_prepare(FullCirc , 2 , np.log(2) , [[1] , [0.5 , 0.5]] , kq_qbits_index , iq_qbits_index , q_qbits_index ,  False)


# Initializing the |z> state:
InitStateZ = Statevector.from_label('0'*number_of_spins)

# Making the full system state:
InitState = Statevector.from_label('0'*Nkq)
InitState = Statevector.from_label('0'*Niq).tensor(InitState)
InitState = InitStateZ.tensor(InitState)
InitState = Statevector.from_label('0'*Qmax).tensor(InitState)
InitState = Statevector.from_label('0').tensor(InitState)

final = InitState.evolve(FullCirc)
finalinds = []
finalstate = []
for i in range(len(final)):
    if abs(final[i]) > 1E-6:
        finalinds.append(i)
        finalstate.append(final[i])
print(f'The final state is {finalstate} and the indices are {finalinds}')

The final state is [(0.7191874465197123+7.151800912622856e-16j), (0.4233892537998104+1.2049272784730999e-17j), (0.4233892537998102+1.2049272784730971e-17j), (0.17624733778282453-1.8466017944796e-16j), (0.17624733778282442-1.6503402371441281e-16j), (0.1762473377828244-2.1409941304828079e-16j), (0.1762473377828243-2.33725568781828e-16j)] and the indices are [0, 68, 69, 204, 205, 206, 207]


/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/qiskit_0.42/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/qiskit_0.42/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [14]:
print(np.sqrt(np.log(2)/2.0)/np.sqrt(1 + np.log(2) + (np.log(2))**2/2.0))

0.4233892537998112
